In [ ]:
# https://github.com/sohaibomr/tweet-sentiment-pyspark/blob/master/tweet_sentiment_analysis.py

In [57]:
from __future__ import print_function

In [58]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import  Vectors as MLLibVectors
from pyspark.mllib.classification import NaiveBayes
from pyspark.ml.feature import HashingTF,IDF

import shutil
import re

In [80]:
def clean_tweet(row):
    """
    description:Clean right,left spaces from the tweet, links and other characters
    and return a tuple(sentiment, [list_of_words])
    """
    split_row=row.split(",")
    sentiment=split_row[8]
    tweet=split_row[7]
    tweet.strip()
    
    #remove url from the tweet
    tweet=re.sub(r"http\S+", "", tweet)
    
    #remove other characters from the tweet
    tweet=re.sub("[^a-zA-z]", " ", tweet)
    
    tweet=tweet.lower()
    #convert tweet in to list of words
    words_list=tweet.split()
    
    return(sentiment, words_list)

In [81]:
# to start a spark context
sc=SparkContext.getOrCreate()   
    
spark = SparkSession.builder.appName("TweetSentiApp").getOrCreate()

In [82]:
#read data file
data=sc.textFile("datasetReviewed.csv")
head=data.first()
data=data.filter(lambda r: r!=head)
dataset=data.map(lambda r: clean_tweet(r))

In [83]:
#convert rdd into sql dataframe to remove stop words, make ngram model and turn review into a feature
dataframe=spark.createDataFrame(dataset, ["sentiment", "tweet"])

In [118]:
#now remove stopwords from the review(list of words) 
remover=StopWordsRemover(inputCol="tweet", outputCol="filtered")
filtered_df=remover.transform(dataframe)

In [119]:
list(filtered_df)

[Column<b'sentiment'>, Column<b'tweet'>, Column<b'filtered'>]

In [170]:
#now make 2-gram model
ngram=NGram(n=2, inputCol="filtered", outputCol="2gram")
gram_df=ngram.transform(filtered_df)

In [175]:
#now make term frequency vectors out of data frame to feed machine
hashingtf=HashingTF(inputCol="2gram", outputCol="tf", numFeatures=2000)
tf_df=hashingtf.transform(gram_df)

In [172]:
print(tf_df['tf'])

Column<b'tf'>


In [ ]:
#tf-idf
tf_df.cache()
idf=IDF(inputCol="tf", outputCol="idftf", minDocFreq=2)
idfModel=idf.fit(tf_df)

In [142]:
#convert dataframe t rdd, to make a LabeledPoint tuple(label, feature, vector) for machine
tf_rdd=tf_df.rdd

In [143]:
#we also need to convert ml.sparsevector mllib.sparse vector, because naive bayes only accepts mllib.sparsevector type
train_dataset=tf_rdd.map(lambda x: LabeledPoint(float(x.sentiment), MLLibVectors.fromML(x.tf)))

In [144]:
#split dataset into train, test
train, test=train_dataset.randomSplit([0.9, 0.1], seed = 11)

In [132]:
print(train.first())
print(test.first())

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 38.0 failed 1 times, most recent failure: Lost task 0.0 in stage 38.0 (TID 52, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-82-6433bc9b7faf>", line 5, in <lambda>
  File "<ipython-input-80-4e0016e973f4>", line 7, in clean_tweet
IndexError: 

In [145]:
#training
print("************************TRAINIG*******************************")
model=NaiveBayes.train(train, 1.0)
print("*****************************TRAINING COMPLETE************************************")

************************TRAINIG*******************************


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 43.0 failed 1 times, most recent failure: Lost task 0.0 in stage 43.0 (TID 61, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-82-6433bc9b7faf>", line 5, in <lambda>
  File "<ipython-input-80-4e0016e973f4>", line 7, in clean_tweet
IndexError: 

In [54]:
#saving the model
output_dir = 'myNaiveBayesModel_Tweet3'
shutil.rmtree(output_dir, ignore_errors=True)
model.save(sc,output_dir)

In [55]:
#testing on test data
print("************************TESTING***********************************")
predictionAndLabel=test.map(lambda x: (x.label, model.predict(x.features)))
accuracy=1.0*predictionAndLabel.filter(lambda x: x[0]==x[1]).count()/test.count()
print("Model Accuracy is ", accuracy)
print("*****************TESTING COMPLETED*****************************")

************************TESTING***********************************
Model Accuracy is  0.6014157071247964
*****************TESTING COMPLETED*****************************


In [56]:
sc.stop()